In [208]:
#Библиотеки
import pandas as pd
import numpy as np
import warnings
import json
import os
warnings.filterwarnings('ignore')

#Открытие json
with open('./resources/data_.json') as fp:
    data = json.load(fp)

In [261]:
aggr = 'ITEMS' #название агрегирующего поля
task = 'TITLE' #поле с названием процесса
n_levels = 5 #число уровней агрегации
select = {'STATUS': 'Статус', #поля для отбора с переводом
          'DURATION': 'Длительность',
          'BEGIN_DATE_FORMAT': 'Начало',
          'CLOSE_DATE_FORMAT': 'Окончание',
          'RESPONSOBLE_FIO': 'Ответственный'
         }
lvl = 0
fields_original = [task] + list(select.keys()) + [aggr]
tab = pd.json_normalize(data,max_level=1)[fields_original]
col_index = [name + f'_{lvl}' for name in fields_original]
fields = col_index + fields_original
tab.columns = col_index

#Изъятие остальных уровней
for lvl in range(n_levels):
    tab = tab.explode(f'{aggr}_{lvl}', ignore_index=True)
    tab = pd.concat([tab, tab[f'{aggr}_{lvl}'].apply(pd.Series)], axis=1)
    if n_levels - lvl == 1:
        fields.remove(aggr)
        fields_original.remove(aggr)
    tab = tab[fields]
    col_index = [name + f'_{lvl+1}' for name in fields_original]
    cols = fields[:-len(fields_original)] + col_index
    tab.columns = cols
    fields = col_index + fields

#Упорядочивание столбцов
col_aggr = [x for x in list(tab.columns) if x[:-2]==aggr] #удаление агрегирующего столбца
tab = tab.drop(col_aggr, axis=1)

col_task = [x for x in list(tab.columns) if x[:-2]==task] #расположение задач в порядке возрастания уровня
col_task.sort()
exclude = list(set(list(tab.columns)) - set(col_task))
tab = tab[col_task + exclude]

#Добавление дополнительных агрегаций
for n in range(n_levels):
    tab_add = tab.copy()
    for col in list(tab_add.columns):
        if int(col[-1])>n:
            tab_add[col] = np.nan
    tab = pd.concat([tab, tab_add], axis=0)
tab = tab.drop_duplicates().replace(np.nan, ' ')

#

,TITLE_0,TITLE_1,TITLE_2,TITLE_3,TITLE_4,TITLE_5,BEGIN_DATE_FORMAT_1,CLOSE_DATE_FORMAT_1,RESPONSOBLE_FIO_4,STATUS_4,...,BEGIN_DATE_FORMAT_0,STATUS_3,RESPONSOBLE_FIO_2,RESPONSOBLE_FIO_3,STATUS_0,DURATION_3,RESPONSOBLE_FIO_1,DURATION_1,STATUS_5,CLOSE_DATE_FORMAT_0
0,УВВ-08-4286-22 от 30.12.2022,Задачи договора,001 Заключение договора на работы по модерниза...,001.01 Согласование ТЗ,,,,,,,...,02.11.2022,Просрочена,Оксана Цымбалист,Оксана Цымбалист,В работе,2 дня,,,,31.01.2024
1,УВВ-08-4286-22 от 30.12.2022,Задачи договора,001 Заключение договора на работы по модерниза...,001.02 Проведение ЦЗК,,,,,,,...,02.11.2022,Просрочена,Оксана Цымбалист,Оксана Цымбалист,В работе,2 дня,,,,31.01.2024
2,УВВ-08-4286-22 от 30.12.2022,Задачи договора,001 Заключение договора на работы по модерниза...,001.03 Согласование служебной записки на прове...,,,,,,,...,02.11.2022,Просрочена,Оксана Цымбалист,Оксана Цымбалист,В работе,5 дней,,,,31.01.2024
3,УВВ-08-4286-22 от 30.12.2022,Задачи договора,001 Заключение договора на работы по модерниза...,001.04 Подготовка документов к закупке,,,,,,,...,02.11.2022,Завершена,Оксана Цымбалист,Оксана Цымбалист,В работе,5 дней,,,,31.01.2024
4,УВВ-08-4286-22 от 30.12.2022,Задачи договора,001 Заключение договора на работы по модерниза...,001.05 Закупка,,,,,,,...,02.11.2022,Новая (не просмотрена),Оксана Цымбалист,Оксана Цымбалист,В работе,15 дней,,,,31.01.2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,,,,,005.06 Закупка,,,,Оксана Цымбалист,Новая (не просмотрена),...,,,,,,,,,,
408,,,,,005.07 Рассмотрение заявок закупки,,,,Оксана Цымбалист,Новая (не просмотрена),...,,,,,,,,,,
409,,,,,005.08 Переторжка,,,,Оксана Цымбалист,Новая (не просмотрена),...,,,,,,,,,,
410,,,,,005.09 Подведение итогов закупки,,,,Оксана Цымбалист,Новая (не просмотрена),...,,,,,,,,,,


In [ ]:
#Добавление отступов
expands = list(pivot.columns)[:(pos+1)]
for k, col in enumerate(expands):
    pivot[col] = np.where(pivot[col] == ' ',
                                        ' ',
                                        '__'*k + pivot[col].astype(str))

#Формирование столбцов с данными:
fields_active = fields_original[1:-1]
for col in fields_active:
    for n in range(n_levels+1):
        pivot[col] = np.where(pivot[f'{task}_{n}'] != '',
                            pivot[f'{col}_{n}'],
                            '')
pivot = pivot[col_task + fields_active].replace(np.nan, ' ')
pivot.columns = [x.replace(f'{task}_', '') for x in list(pivot.columns)]
pivot.rename(columns = select, inplace=True)
pivot